This notebook plots the vector strength for each cell for a range of frequencies and different input configurations.
The code may be superseeded by SAM-VS-plot.py in the vcnmodel/plotters directory. 


In [1]:
%matplotlib qt

from pathlib import Path
import io
import importlib
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import palettable
sys.path.insert(0, os.path.abspath('nb'))
import VS_data_30dB as VS_data
import matplotlib.pyplot as mpl
from matplotlib import scale as mscale
from  pylibrary.plotting import plothelpers as PH
import toml
config = toml.load(open("../wheres_my_data.toml", "r"))
from matplotlib import transforms as mtransforms
from matplotlib import ticker
import plotnine as PN
def reset_style():
#     sns.set_style(rc={"pdf.fonttype": 42})
    mpl.style.use("~/.matplotlib/figures.mplstyle")
        

In [3]:
importlib.reload(VS_data)

datas = VS_data.data

def prepare_data(datas):
    
    sio = io.StringIO(datas)
    df = pd.read_table(sio, sep=",")
    return df

cell_inputs = {'2': 'subthreshold',
               '5': 'subthreshold',
               '6': 'subthreshold',
               "10": "subthreshold", 
               "30": 'subthreshold',
               
               "9":  "one",
               "13": "one",
               "18": "one",
               
               "11": "two",
               "17": "two",
              }

df = prepare_data(datas)

df["strength"] = df['Cell'].apply(lambda x: 'subthreshold' if x in [5, 10, 2, 6, 30] else 'one' if x in [9, 13, 18] else 'two')

df = df.sort_values(["Cell", "frequency", "Configuration"], ascending = (True, True, True))
df = df.reset_index()

reset_style()

# plotwhat = "phasesdtime"
# plotwhat = 'phasesd'
plotwhat = 'VectorStrength'
# plotwhat = 'SpikeCount'
# plotwhat = 'logSpikeCount'

if plotwhat == 'phasesdtime':
    df['phasesdtime'] = 1e3*df['phasesd']/(2*np.pi*df['frequency'])
    df['AN_phasesdtime'] = 1e3*df['AN_phasesd']/(2*np.pi*df['frequency'])

if plotwhat == 'logSpikeCount':
    df['logSpikeCount'] = np.log10(df['SpikeCount'])


In [5]:
"""
Plot selected cells for a poster, 1 column, two rows
"""

class Plot_Strength_Groups():
    def __init__(self, df, ax1, ax2):
        self.df = df
        self.ax1 = ax1
        self.ax2 = ax2

    def _scalefun(self, x):
        return [f"{y:.2f}" for y in x]

    def _getCell(self, cell):
        return(f"BC{int(cell):02d}")

    # df = df.sort_values('Cell')
    def make_plots(self):
        labels=set(self.df['frequency'])
        labels = [str(int(l)) for l in sorted(labels)]
        sizes = (2, 10)
        self.df = self.df[self.df["Configuration"].isin(["all"])]
        self.ax1.set_yscale('log')
        sns.lineplot(data=self.df, 
            x="frequency", y=plotwhat,
            hue="strength", marker='o', markersize=9,
           #             hue_order=['subthreshold', 'one', 'two'],
                        sizes=sizes,
                        size="strength",
                         ax=self.ax1, palette="colorblind",
                    # estimator=None,
        )

        self.ax1.plot(self.df['frequency'][0:7], self.df['AN_VS'][0:7], 'r-', linewidth=1.0)  # only 7 frequencies

        sns.lineplot(data=self.df,
                    x="frequency", y="AN_VS",
                    hue_order=['subthreshold', 'one', 'two'],
                         sizes=sizes,
        #             size="strength",
                     ax=self.ax1, palette="husl",
                    # estimator=None,
        )

        print(self.df['VectorStrength'])
            
        self.ax1.set_xlim(40, 1100)
        
        self.ax1.set_yscale("linear")

        self.ax1.set_ylim((0, 1))

        # ax2.set_yscale('squareroot')
        sns.lineplot(data=self.df, 
            x="frequency", y='phasesd',
            hue="strength", marker='o', markersize=9,
                        hue_order=['subthreshold', 'one', 'two'],
        #                 sizes=sizes,
                        size="strength",
                         ax=self.ax2, palette="colorblind",
                    # estimator=None,
        )


        self.ax2.set_xlim(40, 1100)
  #      self.ax2.set_xscale("log")
        self.ax2.set_ylim((0, 2*np.pi))


f, axs = mpl.subplots(1, 2, figsize=(8, 4))

PSG = Plot_Strength_Groups(df, ax1=axs[0], ax2=axs[1])
PSG.make_plots()
mpl.show()
save_file="Compare_VS_Cells.pdf"
# mpl.savefig(
#     Path(config["baseDataDirectory"], "Figures", save_file),
#     metadata={
#         "Creator": "Paul Manis",
#         "Author": "Paul Manis",
#         "Title": "SBEM Project Figure 6 Modeling : Vector Strength Summary-cells9-10",
#     },
# )

/Users/pbmanis/Desktop/Python/VCNModel/vcn_venv/lib/python3.9/site-packages/seaborn/_core.py:1194: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


0      0.2435
3      0.4239
6      0.8670
9      0.6941
12     0.6360
        ...  
225    0.7122
228    0.6293
231    0.5167
234    0.2269
237    0.0666
Name: VectorStrength, Length: 80, dtype: float64


In [5]:
"""
Plot all the cells in two columns using ggplot
"""

def getCell(cell):
    return(f"BC{int(cell):02d}")

labels=set(df['frequency'])
labels = [str(int(l)) for l in sorted(labels)]

def scalefun(x):
    return [f"{y:.2f}" for y in x]

print(plotwhat)
anpt = "AN_phasesd"
if plotwhat == 'phasesdtime':
    yscale = [0.0, 1.5]
    anpt = 'AN_phasesdtime'
elif plotwhat == 'SpikeCount':
    yscale = [0, 20000]
elif plotwhat == 'logSpikeCount':
    yscale = [0, 5]
elif plotwhat == 'phasesd':
    yscale = [0, 2*np.pi]
    anpt = "AN_phasesd"
else:
    yscale = [-0.1, 1.0]
    anpt = 'AN_VS'

freqs = [50, 100, 200, 300, 400, 500, 750, 1000];

def fscale(x):
    return([str(int(y)) for y in x])
fr = fscale(freqs)
vsscale = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
def vscale(x):
    return([f"{y:.1f}" for y in x])
dodge = PN.positions.position_dodge(width=0.6)
themer = PN.themes.theme
gg = (PN.ggplot(df,
             PN.aes(x='factor(frequency)',
                           y=plotwhat,
                           group='factor(frequency)',
                           color='factor(Configuration)',
                          ))
             + PN.scale_x_discrete(breaks=freqs, labels=fr)
             + PN.scale_y_continuous(breaks=vsscale, labels=vscale(vsscale))
               # + PN.scales.scale_y_log10()

             + PN.geom_point(
                   PN.aes(
                        x='factor(frequency)',
                        y=plotwhat,
                        inherit=True,
#                         size=1.5,
                    ),
                    stroke=0,
                    position=PN.positions.position_dodge2(width=1),
                 #position=PN.positions.position_jitterdodge(jitter_width=1, dodge_width=1),
                    alpha=1, 
                   size=2.5,
             )
              +PN.labs(x="Frequency (Hz)", y="Vector Strength", 
                       color="Synapse Configuration")
               +PN.geom_line(
                  PN.aes(x='factor(frequency)',
                  y=plotwhat,
                  ),
                    position=PN.positions.position_dodge2(width=1),
                    alpha=1,
                    color='grey',
               )
              + PN.geom_line(
                  PN.aes(x='factor(frequency)',
                  y = anpt,
                    ),
                  position = PN.positions.position_dodge2(width=1),
                alpha=1,
                color = 'red',
              )
               + PN.scales.scale_colour_brewer(type="qual", palette="Dark2")
               + PN.facet_wrap('Cell', labeller=getCell, nrow=5, ncol=2, dir='v')
               + PN.theme_minimal() # (style='whitegrid')
      + PN.theme(text=PN.element_text(family='Arial', ))
      + PN.themes.theme(panel_grid_minor = PN.element_blank())
      + PN.themes.theme(panel_grid_major = PN.element_blank())
      + PN.themes.theme(panel_spacing_x = 0.5,)
      + PN.themes.theme(panel_spacing_y = 0.25,)

      + PN.themes.theme(axis_line = PN.element_rect())
      + PN.themes.theme(axis_line = PN.element_rect())
      
      + PN.themes.theme(axis_ticks_length=20,)
      + PN.themes.theme(axis_ticks_length_minor=0,)
      + PN.themes.theme(axis_ticks_length_major=3,)
      + PN.themes.theme(axis_ticks_direction='in')
      + PN.themes.theme(axis_ticks=PN.element_line(size=1),)
      + PN.themes.theme(axis_ticks_pad=20,)
  
      + PN.themes.theme(axis_ticks_major=PN.element_line(color='k'),)
      + PN.themes.theme(axis_ticks_major_x=PN.element_line(size=1), ) 
      + PN.themes.theme(axis_ticks_major_y=PN.element_line(color='k'),)
  
      + PN.themes.theme(axis_ticks_minor=PN.element_line(color='k'),)
      + PN.themes.theme(axis_ticks_minor_x=PN.element_line(size=0),   )
      + PN.themes.theme(axis_ticks_minor_y=PN.element_line(color='k'),)
      
      + PN.themes.theme(axis_text_x=PN.element_text(angle=45))
#       + PN.scale_x_discrete(labels=labels)
      + PN.scales.ylim(yscale)
      + PN.themes.theme(legend_position=(0.5, 0.93), legend_entry_spacing=0.001)
      + PN.theme(figure_size=(6, 8))


      # + PN.scales.scale_y_continuous(breaks=[f"{f:.1f}" for f in [0, 0.2, 0.4 0.6, 0.8, 1.0]]) # scalefun) # PN.scales.number_format(accuracy = 0.01))
      
     )
# print(dir(PN.scales))

fig, P = gg.draw(return_ggplot = True)
PN.options.figure_size = (10, 10)


for i, ax in enumerate(P.axs):
    ax.tick_params(left=True, bottom=True)

save_file = f"Figure7/Figure7_supp/Figure7_Supplemental2_VS.pdf"

# fig.text(
#     0.99,
#     0.99,
#     save_file,  # .replace('_', '\_'),
#     # transform=fig.transFigure,
#     horizontalalignment="right",
#     verticalalignment="top",
# )
mpl.show()
mpl.savefig(
    Path(config["baseDataDirectory"], "Figures", save_file),
    metadata={
        "Creator": "Paul Manis",
        "Author": "Paul Manis",
        "Title": "SBEM Project Figure 7 Modeling : Vector Strength Summary for different input configurations",
    },
)
        
# fig.savefig('/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/Figures/Fig_M6_VS_SAM.pdf')

VectorStrength


/Users/pbmanis/Desktop/Python/VCNModel/vcn_venv/lib/python3.9/site-packages/plotnine/scales/scales.py:29: PlotnineWarning: Scale for 'y' is already present.
Adding another scale for 'y',
which will replace the existing scale.

